In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from scipy.sparse import csr_matrix

In [2]:
def create_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    outfile.close()

def get_importance(gbm, features):
    create_feature_map(features)
    importance = gbm.get_fscore(fmap='xgb.fmap')
    importance = sorted(importance.items(), key=itemgetter(1), reverse=True)
    return importance

def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

def load_sparse_csr(filename):
    loader = np.load(filename)
    return csr_matrix((  loader['data'], loader['indices'], loader['indptr']),
                         shape = loader['shape'])

In [3]:
df_tr = pd.read_hdf('input_hdf/df_train.hdf5','table')
faron_tr = pd.read_csv('input_hdf/faron_train.csv')
#df_te = pd.read_hdf('input_hdf/df_test.hdf5','table')

In [8]:
label = df_tr.Response.values

In [10]:
df_tr = df_tr.drop(['Id','Response'], axis=1)
faron_tr = faron_tr.drop(['Id'], axis=1)
#df_te = df_te.drop(['Id'], axis=1)

In [17]:
df_tr.replace(-999999, np.nan, inplace=True)

In [22]:
df_tr_with_faron = pd.concat([df_tr,faron_tr], axis=1)

In [23]:
df_tr_with_faron.head()

,L0_S1_F27_2,L0_S1_F29_1,L0_S1_F31_2,L0_S1_F31_4,L0_S1_F31_5,L0_S2_F33_1,L0_S2_F35_1,L0_S2_F35_2,L0_S2_F35_3,L0_S2_F35_4,...,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,StartTime,magic1,magic2,magic3,magic4
0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,82.239998,9999999,-2,-2314450,-224451
1,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,1313.119995,2,-1,-2284042,-67530
2,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,1618.699951,1,-2,-2313512,-26876
3,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,1149.199951,2,-2,-2196581,-3564
4,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,602.640015,2,-2,-2226678,-1


In [24]:
df_tr_with_faron.to_hdf('df_tr_with_faron.hdf5', 'table')

In [3]:
df_tr_with_faron = pd.read_hdf('input_hdf/df_tr_with_faron.hdf5','table')

In [4]:
label = pd.read_csv('input_orig/train_numeric.csv', usecols=['Response']).values

In [9]:
dtrain = xgb.DMatrix(df_tr_with_faron, label=label)

In [11]:
create_feature_map(df_tr_with_faron.columns)

In [12]:
dtrain.save_binary("train_with_faron.buffer")

In [ ]:
bst = xgb.Booster() #init model
bst.load_model("0001.model") # load data

from operator import itemgetter
get_importance(bst,df_tr.columns)

In [13]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
udev             52G     0   52G   0% /dev
tmpfs            11G  8.7M   11G   1% /run
/dev/sda1       194G  115G   80G  59% /
tmpfs            52G     0   52G   0% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
tmpfs            52G     0   52G   0% /sys/fs/cgroup
tmpfs            11G   12K   11G   1% /run/user/1001
